## Data cleaning/提取新加坡数据集（无需打开）

设数据集包含以下字段：

job_id：工作岗位的唯一标识符
job_title：工作岗位标题
company：公司名称
location：工作地点
提取工作岗位实体：

读取数据：

使用 Pandas 或其他工具读取工作岗位数据集。




In [3]:
'''
Author: Tianjun Chi harrychi@sjtu.edu.cn
Date: 2024-09-18 11:05:44
LastEditors: Tianjun Chi harrychi@sjtu.edu.cn
LastEditTime: 2024-09-18 11:46:27
FilePath: \practice module\group project\IRS-PM-2024-09-10-GRP2-JobRecommendationSystem\Data_Processing\data_process.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
import pandas as pd

job = pd.read_csv('Dataset\jobDescription\job_descriptions.csv')
job_df = job[job['Country'] == 'Singapore']
print(job_df.shape[0])
'''
row_num = 2000 - job_df.shape[0] # get the row number of the dataset
job_ch = job[job['Country'] == 'China'] # get the rows of the dataset based on the rownum

job_ch = job_ch.sample(n=row_num, random_state=1) 

job_df = pd.concat([job_df, job_ch]) # combine the two dataset
job_df.to_csv('Dataset\jobDescription\job_sgcn.csv', index=False)
'''
print('add')


7425
add


保存新加坡工作子集

In [ ]:
# save to csv
# job_df.to_csv('Dataset\jobDescription\job_sg.csv', index=False)

## 处理新加坡数据

In [55]:
# load dataset
job_df = pd.read_csv('Dataset\jobDescription\job_sg.csv')

In [ ]:
# traverse all the columns and get column list
column_list = job_df.columns
entity_cols = []

print(column_list)
# print the sample value for each column
for column in column_list:
    print(job_df[column].head(2))
    print('-------------------------------------')

experience 

qualifications 

Work Type 

responsibilities 

Company Size

Job Title

Role

Job Portal

Job Description

Benefits

skills



Company Company Profile leave for process

提取实体

根据需要提取的实体。比如，你可能对 job_title、company 和 location 感兴趣。

In [ ]:
entity_jobTitles = job_df['Job Title'].unique()
entity_companies = job_df['Company'].unique()
entity_experience = job_df['Experience'].unique()

print('experience:', len(entity_experience),entity_experience)#, experience)
print('job_titles:', len(entity_jobTitles))#, job_titles)
print('companies:', len(entity_companies))#, companies)

### 预览： 输出需要利用的实体

In [11]:
import pandas as pd

# 读取工作岗位数据集
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')

# 提取工作岗位实体
job_titles = job_df['Job Title'].unique()
job_roles = job_df['Role'].unique()
#companies = job_df['company'].unique()
#locations = job_df['location'].unique()

# 清洗和去重
job_titles = list(set(job_titles))
job_roles = list(set(job_roles))
#companies = list(set(companies))
#locations = list(set(locations))

# 读取应聘者数据集
applicant_df = pd.read_csv('Dataset/user/user_clean.csv')

# 提取应聘者实体
names = applicant_df['Respondent'].unique()
skills = applicant_df['DevType'].str.split(';', expand=True).stack().unique()

# 清洗和去重
names = list(set(names))
skills = set(skill.strip() for skill in skills)

# 整合所有实体
#all_entities = list(set(job_titles + companies + locations + names + list(skills)))

# 输出结果
print("Job Titles:", job_titles)
print("Names:", names)
print("Skills:", skills)
print("roles:", job_roles)
#print("All Entities:", all_entities)


Job Titles: ['Systems Administrator', 'Database Administrator', 'Graphic Designer', 'Data Scientist', 'Environmental Consultant', 'Landscape Architect', 'Software Engineer', 'Technical Writer', 'Physical Therapist', 'Litigation Attorney', 'Nurse Manager', 'Network Analyst', 'IT Manager', 'Speech Therapist', 'HR Coordinator', 'Product Designer', 'Marketing Manager', 'Digital Marketing Specialist', 'Network Engineer', 'Inventory Analyst', 'Accountant', 'Legal Counsel', 'Project Manager', 'IT Administrator', 'Family Nurse Practitioner', 'Financial Advisor', 'Marketing Coordinator', 'Mechanical Designer', 'Web Designer', 'Public Relations Specialist', 'Data Entry Clerk', 'Finance Manager', 'Executive Assistant', 'Landscape Designer', 'Office Manager', 'Teacher', 'Network Technician', 'Architect', 'Systems Analyst', 'Architectural Designer', 'Nurse Practitioner', 'UX/UI Designer', 'Psychologist', 'Mechanical Engineer', 'Financial Analyst', 'Financial Planner', 'Procurement Specialist', 'Eve

## 获得 train list(user-job interaction)



基于TF-IDF的文本相似度计算

get the job recommendations for each user based on job descriptions

version 1: 只利用job title & skills 构建interaction

In [12]:
import pandas as pd
# 读取应聘者&工作数据集
applicant_df = pd.read_csv('Dataset/user/user_clean.csv')
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')

# 提取工作岗位实体
job_titles = job_df['Job Title'].unique()
# remove 'art teacher' from job titles
job_titles = [title for title in job_titles if title != 'Art Teacher']

job_roles = job_df['Role'].unique()

# job titles - DevType


# 提取应聘者实体
names = applicant_df['Respondent'].unique()
skills = applicant_df['DevType'].str.split(';', expand=True).stack().unique()

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
#job_tfidf = vectorizer.fit_transform(list(job_titles[0]))
#skills_tfidf = vectorizer.transform(skills[1])
job_tfidf = vectorizer.fit_transform(job_titles)
job_role_tfidf = vectorizer.transform(job_roles)
skills_tfidf = vectorizer.transform(skills)
#print('job_tfidf: ',job_tfidf)
#print('job_role_tfidf: ',job_role_tfidf)
#print('skills_tfidf: ',skills_tfidf)
# construct a dictionary to store the matching results
matching_results = {}

from sklearn.metrics.pairwise import cosine_similarity

for i in range(0,len(applicant_df)):
    if type(applicant_df['DevType'].iloc[i])!=str:
        print('skip',i)
        continue
    skills = applicant_df['DevType'].iloc[i].split(';') # get the skills list of user i
    # get the tfidf vector of the user's skills
    skills_tfidf = vectorizer.transform(skills)

    # compute the cosine similarity between the user's skills and job descriptions
    similarity_job_skill = cosine_similarity(job_tfidf, skills_tfidf)
    similarity_role_skill = cosine_similarity(job_role_tfidf, skills_tfidf)
    print('len: ',len(similarity_job_skill),len(similarity_job_skill[1]))#,similarity_job_skill)

    # find the matching for candidate skills to job descriptions(20 -> 147) with a threshold of 0.5
    fit = 0
    for k in range(0,len(similarity_job_skill[1])):
        #for j in range(0,len(similarity_job_skill)):
        # get the k-th column
        sim = similarity_job_skill[:,k]
        # rank the similarity score in decresing order and get the index
        #print(sim)
        idx = sim.argsort()[::-1] # decrease order of the similarity score
        for j in range(0,10):
            if sim[idx[j]] > 0.5:
                fit += 1
                print(job_df['Job Title'].iloc[idx[j]],sim[idx[j]])
                # add the result to the dictionary, if not empty, add the job title to the existing list
                job_title = job_df['Job Title'].iloc[idx[j]]
                if matching_results.get(i) == None:
                    matching_results[i] = job_title
                else:
                    matching_results[i] = matching_results[i] +','+ job_title
    
    if fit == 0:
        print('No matching job found for',i)

len:  146 3
Marketing Coordinator 1.0
Urban Planner 0.5843203977152347
Procurement Manager 1.0
Software Engineer 0.5818362013606097
Urban Planner 1.0
Market Research Analyst 0.7764760585827926
Marketing Coordinator 0.5843203977152347
len:  146 3
Marketing Coordinator 1.0
Urban Planner 0.5843203977152347
Environmental Consultant 0.665957844477403
Data Entry Clerk 0.6244550123890088
Software Engineer 0.6244550123890088
Software Engineer 0.6244550123890088
UX/UI Designer 0.5939594418859105
Urban Planner 0.5031328116645648
QA Analyst 0.6562490321439156
Marketing Coordinator 0.6485835542394485
len:  146 3
Marketing Coordinator 1.0
Urban Planner 0.5843203977152347
Environmental Consultant 0.665957844477403
Data Entry Clerk 0.6244550123890088
Software Engineer 0.6244550123890088
Software Engineer 0.6244550123890088
UX/UI Designer 0.5939594418859105
Urban Planner 0.5031328116645648
Environmental Consultant 0.665957844477403
Data Entry Clerk 0.6244550123890088
Software Engineer 0.62445501238900

In [13]:
 matching_results

{0: 'Marketing Coordinator,Urban Planner,Procurement Manager,Software Engineer,Urban Planner,Market Research Analyst,Marketing Coordinator',
 1: 'Marketing Coordinator,Urban Planner,Environmental Consultant,Data Entry Clerk,Software Engineer,Software Engineer,UX/UI Designer,Urban Planner,QA Analyst,Marketing Coordinator',
 2: 'Marketing Coordinator,Urban Planner,Environmental Consultant,Data Entry Clerk,Software Engineer,Software Engineer,UX/UI Designer,Urban Planner,Environmental Consultant,Data Entry Clerk,Software Engineer,Software Engineer,UX/UI Designer,Urban Planner',
 3: 'Marketing Coordinator,Urban Planner,Urban Planner,Market Research Analyst,Marketing Coordinator,Environmental Consultant,Data Entry Clerk,Software Engineer,Software Engineer,UX/UI Designer,Urban Planner,Environmental Consultant,Data Entry Clerk,Software Engineer,Software Engineer,UX/UI Designer,Urban Planner',
 4: 'Marketing Coordinator,Urban Planner',
 5: 'Environmental Consultant,Data Entry Clerk,Software Eng

In [14]:
len(matching_results)

1386

Matching results:将User ID和对应 Job title映射到 Job dataset的 Job Title 构成的字典

In [15]:
# element number of the dictionary
print(len(matching_results))
#print(matching_results)

1386


将上述的 User ID 映射到 Job dataset中的 Job id （一个Job Title 对应多个 Job id）

In [16]:
# 将User ID和对应job title映射到job dataset的job id
# 读取工作岗位数据集
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')
user_job_mapping = {}

for i in range(0,len(matching_results)):
    # jump the user if no matching job found
    if matching_results.get(i) == None:
        print('No result for user',i)
        continue

    # get the job titles of the user i
    job_titles = matching_results[i].split(',')

    # get the job ids of the user i
    job_ids = []
    for job_title in job_titles:

        # random select 15 job ids if there are multiple job titles with the same job title
        for index in job_df[job_df['Job Title'] == job_title].sample(10).index.tolist():
            job_ids.append(index)

    # add the job ids to the dictionary
    user_job_mapping[i] = job_ids
#print(user_job_mapping)


分割训练集和测试集，保存为txt文件格式

In [17]:
print(user_job_mapping.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [18]:
# split the user job mapping into training set and a test set with 7:3
import random
train_user_job_mapping = {}
test_user_job_mapping = {}
# for each user, randomly shuffle the job ids and split them into training set and test set
for user_id, job_ids in user_job_mapping.items():
    random.shuffle(job_ids)
    train_size = int(0.7 * len(job_ids))
    train_job_ids = job_ids[:train_size]
    test_job_ids = job_ids[train_size:]
    # add the training set and test set to the dictionary
    train_user_job_mapping[user_id] = train_job_ids
    test_user_job_mapping[user_id] = test_job_ids


'''
import random
# get the user ids
user_ids = list(user_job_mapping.keys())
# shuffle the user ids
random.shuffle(user_ids)
# get the training set and test set
train_size = int(0.8 * len(user_ids))
train_user_ids = user_ids[:train_size]
test_user_ids = user_ids[train_size:]
# get the training set and test set
train_user_job_mapping = {user_id: user_job_mapping[user_id] for user_id in train_user_ids}
test_user_job_mapping = {user_id: user_job_mapping[user_id] for user_id in test_user_ids}
print('user_job_mapping:',len(user_job_mapping))
print('train_user_job_mapping:',len(train_user_job_mapping))
print('test_user_job_mapping:',len(test_user_job_mapping))
'''
# save the user job mapping to a txt file , each line is a user id and the corresponding job ids
with open('train.txt', 'w') as f:
    for key in train_user_job_mapping.keys():
        # write the user id and job ids to the file, each job id is separated by space
        f.write(f"{key}")
        for job_id in train_user_job_mapping[key]:
            f.write(f" {job_id}")
        f.write('\n')    

with open('test.txt', 'w') as f:
    for key in test_user_job_mapping.keys():
        # write the user id and job ids to the file, each job id is separated by space
        f.write(f"{key}")
        for job_id in test_user_job_mapping[key]:
            f.write(f" {job_id}")
        f.write('\n')    


## 获得 item list

Job Id -> remap Id+Job Id / Job Id -> remap Id

In [ ]:
## map job id to remap id which start from 0 and write into txt file
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')
job_ids = job_df['Job Id']

ct = 0
with open('item_list.txt','w') as f:
    for i in range(0,len(job_ids)):
        f.write(f"{job_ids[i]} {i} {job_ids[i]}\n")
        ct +=1
print(ct)

##  获得entity list 

entity list = job id list + user id list + job attribute list


write item and item attributes into txt file as entity list

提取item所有属性

In [24]:
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')
job_cols = list(job_df.columns)
print(job_cols)
job_cols.remove('Job Id')
job_cols.remove('latitude')
job_cols.remove('longitude')
job_cols.remove('Country')
job_cols.remove('Contact Person')
job_cols.remove('Contact')
job_cols.remove('Job Description')
print(job_cols)
# get all the unique value of the all the cols of the dataset
for col in job_cols:
    job_cols_unique = job_df[col].unique()
    print(col,': ',job_cols_unique)
# Experience
# Qualifications
# Work Type
# Company Size -分类
# Job Title -分割
# skills -分割
# company 
#Job Description
# Company Profile -> Sector Industry 
# get the sector and industry of the company profile from the company profile'{"Sector":"Utilities","Industry":"Utilities: Gas and Electric","City":"Arlington","State":"Virginia","Zip":"22203","Website":"www.aes.com","Ticker":"AES","CEO":"Andres R. Gluski"}'


['Job Id', 'Experience', 'Qualifications', 'Salary Range', 'location', 'Country', 'latitude', 'longitude', 'Work Type', 'Company Size', 'Job Posting Date', 'Preference', 'Contact Person', 'Contact', 'Job Title', 'Role', 'Job Portal', 'Job Description', 'Benefits', 'skills', 'Responsibilities', 'Company', 'Company Profile']
['Experience', 'Qualifications', 'Salary Range', 'location', 'Work Type', 'Company Size', 'Job Posting Date', 'Preference', 'Job Title', 'Role', 'Job Portal', 'Benefits', 'skills', 'Responsibilities', 'Company', 'Company Profile']
Experience :  ['1 to 10 Years' '3 to 9 Years' '2 to 9 Years' '4 to 13 Years'
 '2 to 13 Years' '1 to 14 Years' '1 to 13 Years' '2 to 12 Years'
 '0 to 15 Years' '4 to 15 Years' '1 to 12 Years' '2 to 10 Years'
 '3 to 10 Years' '3 to 14 Years' '3 to 15 Years' '5 to 11 Years'
 '3 to 8 Years' '3 to 11 Years' '0 to 14 Years' '2 to 15 Years'
 '5 to 15 Years' '5 to 14 Years' '0 to 13 Years' '1 to 9 Years'
 '4 to 12 Years' '3 to 12 Years' '0 to 12 Ye

### 1.get job id list

In [62]:
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')
job_cols = list(job_df.columns)
print(job_cols)
job_id_list = set({})
# job_cols.remove('Job Id')
# job_cols.remove('latitude')
# job_cols.remove('longitude')
# job_cols.remove('Country')
# job_cols.remove('Contact Person')
# job_cols.remove('Contact')
# job_cols.remove('Job Description')
# print(job_cols)
for id in job_df['Job Id'].unique():
    job_id_list.add(id)
print(len(job_id_list))



['Job Id', 'Experience', 'Qualifications', 'Salary Range', 'location', 'Country', 'latitude', 'longitude', 'Work Type', 'Company Size', 'Job Posting Date', 'Preference', 'Contact Person', 'Contact', 'Job Title', 'Role', 'Job Portal', 'Job Description', 'Benefits', 'skills', 'Responsibilities', 'Company', 'Company Profile', 'Sector', 'Industry']
7425


### 2. get user id list

In [63]:
user = pd.read_csv('Dataset/user/user_clean.csv')
# get respodent id as user id
resp = user['Respondent'].unique()
user_id_list = set({})
for id in resp:
    user_id_list.add(id)
print(len(user_id_list))

280


### 3. get job attribute list

In [64]:
job_attribute_list = set({})

#### 提取Job Title

In [65]:
# get all the unique value of job title col of the dataset
job_title_list = set({})
job_titles = job_df['Job Title'].unique()
for title in job_titles:
    job_title_list.add(title)

job_attribute_list.update(job_title_list)
job_title_list

{'Account Director',
 'Account Executive',
 'Account Manager',
 'Accountant',
 'Administrative Assistant',
 'Aerospace Engineer',
 'Architect',
 'Architectural Designer',
 'Art Director',
 'Art Teacher',
 'Back-End Developer',
 'Brand Ambassador',
 'Brand Manager',
 'Business Analyst',
 'Business Development Manager',
 'Chemical Analyst',
 'Chemical Engineer',
 'Civil Engineer',
 'Content Writer',
 'Copywriter',
 'Customer Service Manager',
 'Customer Service Representative',
 'Customer Success Manager',
 'Customer Support Specialist',
 'Data Analyst',
 'Data Engineer',
 'Data Entry Clerk',
 'Data Scientist',
 'Database Administrator',
 'Database Developer',
 'Dental Hygienist',
 'Digital Marketing Specialist',
 'Electrical Designer',
 'Electrical Engineer',
 'Email Marketing Specialist',
 'Environmental Consultant',
 'Environmental Engineer',
 'Event Coordinator',
 'Event Manager',
 'Event Planner',
 'Executive Assistant',
 'Family Lawyer',
 'Family Nurse Practitioner',
 'Finance Mana

#### 提取 Work Type

In [66]:
# get Work Type info
work_type_list = set({})
work_type = job_df['Work Type'].unique()
for work in work_type:
    work_type_list.add(work)

job_attribute_list.update(work_type_list)
work_type_list

{'Contract', 'Full-Time', 'Intern', 'Part-Time', 'Temporary'}

#### 提取 Company

In [67]:
# get company name
company_list = set({})
company = job_df['Company'].unique()
for comp in company:
    company_list.add(comp)
print(len(company_list))

job_attribute_list.update(company_list)
company_list


888


{'3M',
 '3i Group',
 'A-Mark Precious Metals',
 'ABM Industries',
 'AECOM',
 'AES',
 'AGCO',
 'AMP Limited',
 'APA',
 'APA Group',
 'ARKO',
 'AT&T',
 'AbbVie',
 'Abbott Laboratories',
 'Activision Blizzard',
 'Adani Enterprises',
 'Adani Green Energy',
 'Adani Ports and Special Economic Zone',
 'Adelaide Brighton',
 'Adidas AG',
 'Aditya Birla Capital',
 'Admiral Group',
 'Adobe',
 'Advance Auto Parts',
 'Advanced Micro Devices',
 'Aflac',
 'Agricultural Bank of China',
 'Air Products & Chemicals',
 'Airbnb',
 'Airtel Africa',
 'Alaska Air Group',
 'Albemarle',
 'Albertsons',
 'Alcoa',
 'Alibaba Group',
 'Alibaba Health Information Technology',
 'Allianz SE',
 'Allstate',
 'Ally Financial',
 'Alphabet',
 'Altice USA',
 'Altria Group',
 'Alumina Limited',
 'Amazon.com',
 'Ambuja Cements',
 'Ameren',
 'American Airlines Group',
 'American Electric Power',
 'American Express',
 'American Family Insurance Group',
 'American International Group',
 'American Tower',
 'Ameriprise Financial',


#### 处理Company Profile，提取字符串中的Sector和Industry数据作为Entity

In [68]:
# get the sector and industry of the company profile 
import json
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')
company_profile_list = set({})
job_cols_unique = job_df['Company Profile'].unique()

substr_ceo = "CEO"
for profile in job_cols_unique:
    # get the row index of company profile 
    idx = job_df[job_df['Company Profile'] == profile].index
    
    #print(profile)
    #print(type(profile))
    # check if the profile is a string
    if isinstance(profile, str):
        profile = profile[:profile.index(substr_ceo)-2]+'}'
        #print(profile)
        profile = json.loads(str(profile))

        # remove the info aftering 'CEO' that cannot be splitted
        company_profile_list.add(profile['Sector'])
        company_profile_list.add(profile['Industry'])
        # add the sector and industry into the same row as company file in the dataset, as two extra columns
        for idx in idx:
            # add the sector information into the dataset as a new column Sector and row index idx
            job_df.loc[idx, 'Sector'] = profile['Sector']
            job_df.loc[idx, 'Industry'] = profile['Industry']

# save the dataset with the new columns
#job_df.to_csv('Dataset/jobDescription/job_sg.csv', index=False)
job_attribute_list.update(company_profile_list)
company_profile_list

{'Advertising & Marketing',
 'Advertising and Marketing',
 'Advertising, Marketing',
 'Aerospace & Defense',
 'Aerospace and Defense',
 'Aerospace/Building Systems',
 'Aerospace/Defense',
 'Agriculture',
 'Agrochemicals',
 'Airlines',
 'Airlines & Aviation',
 'Aluminum',
 'Aluminum and Copper Manufacturing',
 'Apparel',
 'Apparel & Footwear',
 'Apparel and Footwear',
 'Apparel and Textiles',
 'Apparel/Fashion',
 'Appliances',
 'Appliances and Electronics',
 'Asset Management',
 'Automotive',
 'Automotive Parts',
 'Automotive Retailing, Services',
 'Automotive/Motorcycles',
 'Aviation and Travel',
 'Aviation/Airlines',
 'Banking',
 'Banking & Financial Services',
 'Banking and Financial Services',
 'Banking/Financial Services',
 'Beauty',
 'Beauty/Cosmetics',
 'Beverage',
 'Beverage/Alcohol',
 'Beverages',
 'Biotechnology',
 'Building Materials',
 'Building Materials & Equipment',
 'Building Materials, Glass',
 'Building Materials/Construction',
 'Business Services',
 'Business Services

In [69]:
job_df['Sector'].unique()

array(['Utilities', 'Conglomerate', 'Healthcare', 'Mining and Metals',
       'Technology', 'Industrial', 'Telecommunications',
       'Aerospace/Building Systems', 'Industrials',
       'Electrical Equipment', 'Financial Services',
       'Media and Entertainment', 'Real Estate', 'Energy', 'Gambling',
       'Cement & Aggregates', 'Transportation', 'Retail',
       'Consumer Goods/Homecare', 'Logistics', 'Insurance',
       'Semiconductors', 'Engineering', 'Technology and Security',
       'Automotive', 'Testing and Certification', 'Travel/Cruise Lines',
       'E-commerce', 'Chemicals', 'Banking', 'Energy/Infrastructure',
       'Restaurants', 'Automotive Parts', nan, 'Entertainment',
       'Packaging', 'Aerospace and Defense', 'Pharmaceuticals', 'Tobacco',
       'Beverage', 'Media', 'Technology/Software', 'Food and Beverage',
       'Business Services', 'Infrastructure', 'Renewable Energy',
       'Communication Equipment', 'Semiconductor', 'Rental Cars',
       'Travel & Hospital

#### 提取experience的value作为Entity的一部分

In [70]:
# 提取experience的value作为Entity的一部分
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')
experience_list = set({})
experience = job_df['Experience'].unique()
for exp in experience:
    experience_list.add(exp)
print(len(experience_list))
job_attribute_list.update(experience_list)

48


In [71]:
len(job_attribute_list)

1434

#### 提取Qualifications的value作为Entity的一部分

In [72]:
# 提取Qualifications的value作为Entity的一部分
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')
# get unique value of the Qualifications
job_cols_unique = job_df['Qualifications'].unique()
print(job_cols_unique)

qualification_list = set({})
for qualification in job_cols_unique:
    if isinstance(qualification, str):
        qualification_list.add(qualification)

job_attribute_list.update(qualification_list)
qualification_list

['MBA' 'B.Com' 'BBA' 'MCA' 'M.Tech' 'BCA' 'B.Tech' 'M.Com' 'PhD' 'BA']


{'B.Com', 'B.Tech', 'BA', 'BBA', 'BCA', 'M.Com', 'M.Tech', 'MBA', 'MCA', 'PhD'}

In [73]:
print(len(job_attribute_list))

1444


#### 提取benefits的value作为Entity的一部分

In [119]:
# 提取benefits词条作为Entity的一部分
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')

benefit_set = set({})
# get unique value of the Benefits
benefits = job_df['Benefits']
for benefit_list in benefits:
    if isinstance(benefit_list, str):
        benefit_list = benefit_list[2:-2]
        benefit_list = benefit_list.split(',')
        for benefit in benefit_list:
            benefit_set.add(benefit)



job_attribute_list.update(benefit_set)
benefit_set

{' Bonuses and Incentive Programs',
 ' Casual Dress Code',
 ' Childcare Assistance',
 ' Employee Assistance Programs (EAP)',
 ' Employee Discounts',
 ' Employee Recognition Programs',
 ' Employee Referral Programs',
 ' Financial Counseling',
 ' Flexible Spending Accounts (FSAs)',
 ' Flexible Work Arrangements',
 ' Health Insurance',
 ' Health and Wellness Facilities',
 ' Legal Assistance',
 ' Life and Disability Insurance',
 ' Paid Time Off (PTO)',
 ' Parental Leave',
 ' Professional Development',
 ' Profit-Sharing',
 ' Relocation Assistance',
 ' Retirement Plans',
 ' Social and Recreational Activities',
 ' Stock Options or Equity Grants',
 ' Transportation Benefits',
 ' Tuition Reimbursement',
 ' Wellness Programs',
 'Casual Dress Code',
 'Childcare Assistance',
 'Employee Assistance Programs (EAP)',
 'Employee Referral Programs',
 'Flexible Spending Accounts (FSAs)',
 'Health Insurance',
 'Legal Assistance',
 'Life and Disability Insurance',
 'Transportation Benefits',
 'Tuition Reim

In [121]:
len(benefit_set)

35

#### 将skill-list作为entities，为后续的KG构建做准备

In [74]:
skill_list = [
        "Python", "JavaScript", "Java", "C","C++", "R", "Ruby", "Swift", "TypeScript", "Go", "PHP", #"programming_language_skills"

        "Machine Learning", "Deep Learning", "ML","DL","Natural Language Processing","NLP","Data Science","Data Mining",#"data_science_skills"
        "Statistical Analysis", "Predictive Modeling", "Data Visualization", "Time Series Analysis",
        "Data Wrangling", "TensorFlow", "Scikit-learn", "PyTorch",  

        "SQL", "Apache Hadoop", "Apache Spark", "ETL","Spark", # "data_processing_skills": 
        "Data Cleaning", "Data Aggregation", "Data Pipelines", "Excel", "Pandas", "Numpy",

        "HTML5", "CSS3", "React.js", "Angular", "Vue.js", "Responsive Web Design", # "front_end_skills": 
        "Bootstrap", "jQuery",
   
        "Node.js", "Django", "Flask", "Ruby on Rails", "Express.js", "Spring Boot", ".NET Core",# "back_end_skills": 
        "RESTful API Development", "GraphQL", "Docker", "Kubernetes", "MongoDB", "MySQL", "PostgreSQL", "Redis",
        
        "Linear Regression", "Logistic Regression", "Decision Trees", "Random Forest", "Support Vector Machines","SVM",# "machine_learning_algorithm_skills"
        "K-Nearest Neighbors", "KNN", "Naive Bayes", "K-Means Clustering", "Hierarchical Clustering", "Principal Component Analysis","PCA",
        "Gradient Boosting", "AdaBoost", "XGBoost", "LightGBM", "Neural Networks", "Convolutional Neural Networks","CNN",
        "Recurrent Neural Networks","RNN", "Long Short-Term Memory","LSTM", "Generative Adversarial Networks","GANs",
        "Reinforcement Learning", "Bayesian Networks"
]
skill_set = set({})
for skill in skill_list:
    if isinstance(skill, str):
        skill_set.add(skill)

job_attribute_list.update(skill_set)

In [120]:
print(len(job_attribute_list))

1568


#### merge multiple sets into one set

In [122]:
# merge multiple sets into one set
all_entities = set({})
all_entities.update(job_id_list)
all_entities.update(user_id_list)
# merge job attributes
all_entities.update(job_attribute_list)
print(len(all_entities))



9273


#### store the entities into a entity.txt file

In [123]:
# store the entities into a entity.txt file
with open('entity_list.txt','w') as f:
    #for entity in all_entities:
    #    f.write(f"{entity}\n")
    # store id and the entity value
    for entity_id,entity in enumerate(all_entities):
        f.write(f"{entity_id} {entity}\n")

## 构建relationship list

requires -> 0  
belongs to -> 1  
hires -> 2  
has -> 3    

### **Relationship type**

1.1 job title -[requires]-> qualifications.  
1.2 job title -[requires]-> skills.  
1.3 job title -[requires]-> experience.  

2.1 Company name -[belongs to]-> industry.  

3.1 Company name -[hires]-> job titles.  

4.1 Company name -[has]-> benefits.  
4.2 Company name [-has]-> sector.  


### 1.requires关系

In [ ]:
# create a dictionary to store the relationship
relationship = {}

#### 1.构建job title [requires] qualifications.的关系。

entity id + relationship id(0) + entity id

In [125]:
# get the entity id of an entity in the entity_list.txt 
def get_entity_id(entity_name, entity_file_path):
    # Open the entity list file and search for the entity name
    with open(entity_file_path, 'r') as file:
        for line in file:
            # Split the line into ID and name (assuming tab or space-separated values)
            entity_id, entity = line.strip().split(maxsplit=1)
            
            # If the entity name matches, return the ID
            if entity == entity_name:
                return entity_id
    return None 

entity_name = "MBA"
entity_file_path = "entity_list.txt"
entity_id = get_entity_id(entity_name, entity_file_path)
print(entity_id,' for ',entity_name)

26  for  MBA


In [126]:
# 构建job title [requires] qualifications.的关系。
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')

# get the job title and qualifications from the job description
job_titles = job_df['Job Title']
qualifications = job_df['Qualifications']


# iterate through the job titles and qualifications
for job_title, qualification in zip(job_titles, qualifications):
    # split the qualifications into a list
    qualification_list = qualification.split(',')
    job_title_id = get_entity_id(job_title, "entity_list.txt")
    # iterate through the qualifications
    for qual in qualification_list:
        # add the relationship to the dictionary
        entity_id = get_entity_id(qual, "entity_list.txt")
        if relationship.get(job_title_id) == None:
            relationship[job_title_id] = [entity_id]
        else:
            relationship[job_title_id].append(entity_id)
    
# print the relationship    
relationship



{'1393': ['26',
  '6740',
  '1319',
  '2632',
  '6457',
  '2632',
  '6718',
  '6457',
  '2632',
  '1579',
  '1579',
  '6457',
  '6740',
  '1579',
  '1446',
  '1885',
  '6718',
  '1579',
  '6521',
  '1885',
  '1319',
  '6718',
  '6457',
  '6457',
  '1579',
  '26',
  '1579',
  '26',
  '6740',
  '6521',
  '6718',
  '26',
  '1446',
  '1446',
  '26',
  '1579',
  '6740',
  '6740',
  '1446',
  '2632',
  '6718',
  '2632',
  '1319',
  '1319',
  '6740',
  '1319',
  '6718',
  '1579',
  '6740',
  '6457',
  '6521',
  '1885',
  '6718',
  '1446',
  '1885',
  '6718',
  '1446',
  '2632',
  '6457',
  '6521',
  '1446',
  '6718',
  '1579',
  '2632',
  '6521',
  '1446',
  '6457',
  '1446',
  '1885',
  '6457',
  '6521',
  '1446',
  '6521',
  '6718',
  '26',
  '1319',
  '26',
  '1319'],
 '6977': ['1319',
  '1885',
  '6457',
  '1579',
  '1319',
  '2632',
  '6718',
  '6457',
  '26',
  '6740',
  '1579',
  '2632',
  '1319',
  '6740',
  '1446',
  '6718',
  '6718',
  '6718',
  '1885',
  '6718',
  '2632',
  '26',
 

In [127]:
len(relationship)

147

#### 2.构建job title [requires] skills的关系。

In [128]:
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')

# get the job title and qualifications from the job description
job_titles = job_df['Job Title']
job_skills = job_df['skills']   
# construct skill list
skill_list = [
        "Python", "JavaScript", "Java","C++", "Ruby", "Swift", "TypeScript", "Go", "PHP", #"programming_language_skills" (no C/R)

        "Machine Learning", "Deep Learning", "ML","DL","Natural Language Processing","NLP","Data Science","Data Mining",#"data_science_skills"
        "Statistical Analysis", "Predictive Modeling", "Data Visualization", "Time Series Analysis",
        "Data Wrangling", "TensorFlow", "Scikit-learn", "PyTorch",  

        "SQL", "Apache Hadoop", "Apache Spark", "ETL","Spark", # "data_processing_skills": 
        "Data Cleaning", "Data Aggregation", "Data Pipelines", "Excel", "Pandas", "Numpy",

        "HTML5", "CSS3", "React.js", "Angular", "Vue.js", "Responsive Web Design", # "front_end_skills": 
        "Bootstrap", "jQuery",
   
        "Node.js", "Django", "Flask", "Ruby on Rails", "Express.js", "Spring Boot", ".NET Core",# "back_end_skills": 
        "RESTful API Development", "GraphQL", "Docker", "Kubernetes", "MongoDB", "MySQL", "PostgreSQL", "Redis",
        
        "Linear Regression", "Logistic Regression", "Decision Trees", "Random Forest", "Support Vector Machines","SVM",# "machine_learning_algorithm_skills"
        "K-Nearest Neighbors", "KNN", "Naive Bayes", "K-Means Clustering", "Hierarchical Clustering", "Principal Component Analysis","PCA",
        "Gradient Boosting", "AdaBoost", "XGBoost", "LightGBM", "Neural Networks", "Convolutional Neural Networks","CNN",
        "Recurrent Neural Networks","RNN", "Long Short-Term Memory","LSTM", "Generative Adversarial Networks","GANs",
        "Reinforcement Learning", "Bayesian Networks"
]


# iterate through the job titls and skills, and judge whether the skills are in skill list, if in, add the relationship
for job_title, job_skill in zip(job_titles, job_skills):


    job_title_id = get_entity_id(job_title, "entity_list.txt")

    # judge whether the skills in skill list are inside the job_skill string
    for skill in skill_list:
        if skill in job_skill:
            entity_id = get_entity_id(skill, "entity_list.txt")
            if relationship.get(job_title_id) == None:
                relationship[job_title_id] = [entity_id]
            else:
                # if the entity id is not in the list, add it
                if entity_id not in relationship[job_title_id]:
                    relationship[job_title_id].append(entity_id)

    
# print the relationship    
relationship


{'1393': ['26',
  '6740',
  '1319',
  '2632',
  '6457',
  '2632',
  '6718',
  '6457',
  '2632',
  '1579',
  '1579',
  '6457',
  '6740',
  '1579',
  '1446',
  '1885',
  '6718',
  '1579',
  '6521',
  '1885',
  '1319',
  '6718',
  '6457',
  '6457',
  '1579',
  '26',
  '1579',
  '26',
  '6740',
  '6521',
  '6718',
  '26',
  '1446',
  '1446',
  '26',
  '1579',
  '6740',
  '6740',
  '1446',
  '2632',
  '6718',
  '2632',
  '1319',
  '1319',
  '6740',
  '1319',
  '6718',
  '1579',
  '6740',
  '6457',
  '6521',
  '1885',
  '6718',
  '1446',
  '1885',
  '6718',
  '1446',
  '2632',
  '6457',
  '6521',
  '1446',
  '6718',
  '1579',
  '2632',
  '6521',
  '1446',
  '6457',
  '1446',
  '1885',
  '6457',
  '6521',
  '1446',
  '6521',
  '6718',
  '26',
  '1319',
  '26',
  '1319'],
 '6977': ['1319',
  '1885',
  '6457',
  '1579',
  '1319',
  '2632',
  '6718',
  '6457',
  '26',
  '6740',
  '1579',
  '2632',
  '1319',
  '6740',
  '1446',
  '6718',
  '6718',
  '6718',
  '1885',
  '6718',
  '2632',
  '26',
 

In [129]:
len(relationship)

147

#### 3.构建job title [requires] experience的关系。

In [130]:
# 构建job title [requires] qualifications.的关系。
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')

# get the job title and qualifications from the job description
job_titles = job_df['Job Title']
experience = job_df['Experience']


# iterate through the job titles and qualifications
for job_title, exp in zip(job_titles, experience):

    job_title_id = get_entity_id(job_title, "entity_list.txt")
    # add the experience relationship to the dictionary
    entity_id = get_entity_id(exp, "entity_list.txt")
    if relationship.get(job_title_id) == None:
        relationship[job_title_id] = [entity_id]
    else:
        relationship[job_title_id].append(entity_id)

# print the relationship
relationship


{'1393': ['26',
  '6740',
  '1319',
  '2632',
  '6457',
  '2632',
  '6718',
  '6457',
  '2632',
  '1579',
  '1579',
  '6457',
  '6740',
  '1579',
  '1446',
  '1885',
  '6718',
  '1579',
  '6521',
  '1885',
  '1319',
  '6718',
  '6457',
  '6457',
  '1579',
  '26',
  '1579',
  '26',
  '6740',
  '6521',
  '6718',
  '26',
  '1446',
  '1446',
  '26',
  '1579',
  '6740',
  '6740',
  '1446',
  '2632',
  '6718',
  '2632',
  '1319',
  '1319',
  '6740',
  '1319',
  '6718',
  '1579',
  '6740',
  '6457',
  '6521',
  '1885',
  '6718',
  '1446',
  '1885',
  '6718',
  '1446',
  '2632',
  '6457',
  '6521',
  '1446',
  '6718',
  '1579',
  '2632',
  '6521',
  '1446',
  '6457',
  '1446',
  '1885',
  '6457',
  '6521',
  '1446',
  '6521',
  '6718',
  '26',
  '1319',
  '26',
  '1319',
  '1600',
  '6937',
  '8510',
  '2293',
  '2061',
  '762',
  '6540',
  '6887',
  '5701',
  '7985',
  '116',
  '3281',
  '762',
  '3281',
  '116',
  '982',
  '8386',
  '1600',
  '663',
  '8510',
  '2142',
  '982',
  '6217',
  '

In [131]:
len(relationship)

147

In [132]:
# for each relation in the relationship, write the relation as job title id + relationship id + qualification id as a line in the relation.txt file
with open('relation.txt', 'w') as f:
    for job_title_id in relationship.keys():
        for entity_id in relationship[job_title_id]:
            f.write(f"{job_title_id} 0 {entity_id}\n")

### 2. Belongs to关系

#### 1.构建Company name[belongs to] industry的关系

entity id + relationship id(1) + entity id

In [133]:
belong_relation = {}

In [134]:
# 构建job title [requires] qualifications.的关系。
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')

# get the job title and qualifications from the job description
company_name = job_df['Company']
industry = job_df['Industry']




# iterate through the job titles and qualifications
for company, industry_l in zip(company_name, industry):
    if isinstance(industry_l, str):
        # split the qualifications into a list
        industry_list = industry_l.split(',')
        company_id = get_entity_id(company, "entity_list.txt")
        # iterate through the qualifications
        for ind in industry_list:
            # add the relationship to the dictionary
            entity_id = get_entity_id(ind, "entity_list.txt")
            if entity_id == None:
                continue
            if belong_relation.get(company_id) == None:
                belong_relation[company_id] = [entity_id]
            else:
                belong_relation[company_id].append(entity_id)
                
# print the relationship    
belong_relation


{'5533': ['6270',
  '6270',
  '6270',
  '6270',
  '6270',
  '6270',
  '6270',
  '6270',
  '6270',
  '6270',
  '6270',
  '6270',
  '6270',
  '6270'],
 '1698': ['5469',
  '5469',
  '5469',
  '5469',
  '5469',
  '5469',
  '5469',
  '5469',
  '5469',
  '5469',
  '5469',
  '5469'],
 '4610': ['7234',
  '7234',
  '7234',
  '7234',
  '7234',
  '7234',
  '7234',
  '7234',
  '7234',
  '7234',
  '7234'],
 '5028': ['6293',
  '6293',
  '6293',
  '6293',
  '6293',
  '6293',
  '6293',
  '6293',
  '6293',
  '6293',
  '6293',
  '6293',
  '6293',
  '6293'],
 '3865': ['2552',
  '2552',
  '2552',
  '2552',
  '2552',
  '2552',
  '2552',
  '2552',
  '2552',
  '2552',
  '2552',
  '2552',
  '2552'],
 '2583': ['98', '98', '98', '98', '98', '98'],
 '4774': ['2773', '2773', '2773', '2773', '2773', '2773', '2773'],
 '3151': ['5295',
  '5295',
  '5295',
  '5295',
  '5295',
  '5295',
  '5295',
  '5295',
  '5295'],
 '5343': ['98', '98', '98', '98', '98', '98', '98', '98', '98'],
 '1299': ['6735', '6735', '6735', '67

In [135]:
# for each relation in the relationship, write the relation as job title id + relationship id + qualification id as a line in the relation.txt file
with open('relation.txt', 'a') as f:
    for job_title_id in belong_relation.keys():
        for entity_id in belong_relation[job_title_id]:
            f.write(f"{job_title_id} 1 {entity_id}\n")

### 3. hire 关系

In [136]:
hire_relation = {}

#### 1.构建company name[hires]job titles的关系。

In [137]:
# 构建job title [requires] qualifications.的关系。
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')

# get the job title and qualifications from the job description
company_name = job_df['Company']
titles = job_df['Job Title']

# iterate through the job titles and qualifications
for company, title in zip(company_name, titles):
    if isinstance(title, str):

        company_id = get_entity_id(company, "entity_list.txt")
        entity_id = get_entity_id(title, "entity_list.txt")
        if company_id == None or entity_id == None: # no search result in entity list
            continue

        if hire_relation.get(company_id) == None:
            hire_relation[company_id] = [entity_id]
        else:
            hire_relation[company_id].append(entity_id)

# print the relationship    
hire_relation

{'5533': ['1393',
  '6977',
  '6135',
  '8626',
  '6392',
  '1010',
  '6372',
  '1518',
  '1010',
  '1812',
  '8055',
  '6249',
  '4844',
  '6716'],
 '1698': ['6977',
  '3621',
  '5928',
  '4884',
  '3883',
  '9097',
  '7462',
  '3618',
  '3537',
  '4070',
  '921',
  '3053'],
 '4610': ['5395',
  '6563',
  '3582',
  '1518',
  '6716',
  '3618',
  '5475',
  '6716',
  '3200',
  '1897',
  '5912'],
 '5028': ['1327',
  '6977',
  '921',
  '2903',
  '3200',
  '6914',
  '6781',
  '5523',
  '8985',
  '3621',
  '5726',
  '8613',
  '8318',
  '8061'],
 '3865': ['6372',
  '8472',
  '7841',
  '7551',
  '5590',
  '2643',
  '1263',
  '1518',
  '8472',
  '3254',
  '5726',
  '1117',
  '7549'],
 '2583': ['1606', '3551', '80', '6176', '5320', '7633'],
 '4774': ['2136', '6135', '2487', '8302', '4111', '2582', '6249'],
 '3151': ['4070',
  '6977',
  '8985',
  '3883',
  '2136',
  '8031',
  '1057',
  '6977',
  '6977'],
 '5343': ['7262',
  '8600',
  '6914',
  '1057',
  '8318',
  '2487',
  '921',
  '8061',
  '7100

In [138]:
# for each relation in the relationship, write the relation as job title id + relationship id + qualification id as a line in the relation.txt file
with open('relation.txt', 'a') as f:
    for job_title_id in hire_relation.keys():
        for entity_id in hire_relation[job_title_id]:
            f.write(f"{job_title_id} 2 {entity_id}\n")

### 4. Have 关系

In [139]:
has_relation = {}

#### 4.1 构建Company name[has]benefits的关系

In [140]:
# 构建Company name[has]benefits的关系
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')

# get the job title and qualifications from the job description
company_name = job_df['Company']
benefits = job_df['Benefits']

# iterate through the job titles and qualifications
for company, benefit_list in zip(company_name, benefits):

    if isinstance(benefit_list, str): # split the benefits by ','
        benefit_list = benefit_list[2:-2] # delete the {' and '} from the beginning and the end of the string
        benefit_list = benefit_list.split(',')
        for benefit in benefit_list:
            company_id = get_entity_id(company, "entity_list.txt")
            entity_id = get_entity_id(benefit, "entity_list.txt")
            if company_id == None or entity_id == None: # no search result in entity list
                continue

            if has_relation.get(company_id) == None:
                has_relation[company_id] = [entity_id]
            else:
                has_relation[company_id].append(entity_id)

# print the relationship    
has_relation

{'5533': ['889',
  '6242',
  '6736',
  '889',
  '6736',
  '2848',
  '889',
  '6242',
  '6736',
  '2603',
  '6242',
  '2603',
  '1654',
  '1260'],
 '1698': ['1654',
  '6382',
  '6242',
  '7907',
  '1260',
  '6382',
  '6382',
  '6242',
  '1260',
  '6242',
  '6736',
  '2848'],
 '4610': ['2603',
  '1873',
  '2848',
  '2603',
  '6736',
  '7907',
  '6242',
  '2603',
  '2603',
  '6382',
  '1260'],
 '5028': ['2848',
  '6382',
  '6242',
  '2848',
  '1873',
  '6242',
  '2848',
  '6382',
  '1654',
  '7907',
  '1260',
  '2603',
  '1873',
  '2848'],
 '3865': ['6242',
  '6382',
  '6382',
  '1873',
  '1654',
  '6382',
  '1873',
  '6382',
  '1873',
  '1260',
  '2603',
  '6382',
  '889'],
 '2583': ['6242', '2603', '1260', '889', '1260', '1654'],
 '4774': ['2603', '1654', '1654', '6242', '6242', '6736', '6736'],
 '3151': ['1654',
  '2603',
  '6242',
  '2848',
  '6242',
  '6736',
  '6382',
  '6382',
  '1873'],
 '5343': ['1873',
  '889',
  '7907',
  '2603',
  '1260',
  '1654',
  '1260',
  '2603',
  '7907'

#### 4.2 构建Company name[has]sector的关系

In [141]:
# 构建Company name[has]benefits的关系
import pandas as pd
job_df = pd.read_csv('Dataset/jobDescription/job_sg.csv')

# get the job title and qualifications from the job description
company_name = job_df['Company']
sectors = job_df['Sector']

# iterate through the job titles and qualifications
for company, sector in zip(company_name, sectors):
    if isinstance(sector, str):

        company_id = get_entity_id(company, "entity_list.txt")
        entity_id = get_entity_id(sector, "entity_list.txt")
        if company_id == None or entity_id == None: # no search result in entity list
            continue

        if has_relation.get(company_id) == None:
            has_relation[company_id] = [entity_id]
        else:
            has_relation[company_id].append(entity_id)
 
#has_relation

In [142]:
# for each relation in the relationship, write the relation as job title id + relationship id + qualification id as a line in the relation.txt file
with open('relation.txt', 'a') as f:
    for job_title_id in has_relation.keys():
        for entity_id in has_relation[job_title_id]:
            f.write(f"{job_title_id} 3 {entity_id}\n")

----------------------------------------------------------------------------------------------------------------------------------------------

# END OF CODE